# 2018-06-08 - Comparaison modèle/humain
# Pour génerer les MC  : 2018-06-17 Notebook

# Le Ring

In [5]:
#freeze layers after and including freezing_layer+1 (layers start at 0)
def freeze_layers(freezing_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if  count < freezing_layer+1 : #to freeze at iteration 1
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#freeze all the layers except the unfrozen one
def freeze_all_layers(unfrozen_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if count == unfrozen_layer :
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#unfreeze every layers
def layers_microwave(nn_model = model) :
    for count,child in enumerate(model.children()) :
        for param in child.parameters():
            param.requires_grad = True

In [6]:
"""
Created on Tue May 15 11:25:41 2018

@author: hugo
"""
from torch.autograd import Variable
import torch 
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F

#Transform
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#Train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=8, shuffle=True,
                                             num_workers=1)

#Test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=8,shuffle=False,
                                             num_workers=1)

###################################################################################

# Hyper-parameters
sequence_length = 49
input_size = 49
hidden_size = 128
num_layers = 2
num_classes = 16
batch_size = 8
num_epochs = 60
learning_rate = 0.003

###################################################################################

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout = 0.3)
        self.conv1 = nn.Conv2d(1,8,30)
        self.conv2 = nn.Conv2d(1,8,30)
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(2,2)
        
        #self.drop = nn.Dropout(0.2)

        self.fc = nn.Linear(113 , 16)  # 2 for bidirection
    
    def forward(self, x):
        #print('Input %s' % str(x.size()))

        out = self.pool1(F.relu(self.conv1(x)))
        #print('Convoluted %s' % str(x.size()))

        # Init
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        h0 = Variable(h0).cuda()
        c0 = Variable(c0).cuda()
        
        out = out[:,-1,:,:]
        #print('Resized for RNN %s' % str(x.size()))
        
        # LSTM forward
        out, _ = self.rnn(out, (h0,c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        #print('After RNN %s' % str(out.size()))
        
        out = out.unsqueeze(1)
        #print(out.size())
        out = self.pool2(F.relu(self.conv2(out)))
        #print(out.size())
        out = out[:,-1,:,:]
        #Dropout
        #out = self.drop(out)
        #print(out.size())
        
        out = out[:,-1,:]
        # LSTM output
        out = self.fc(out)
        #print('Reshaped for output %s \n'%  str(out.size()))

        return out

model = BiRNN(input_size, hidden_size, num_layers, num_classes).cuda()
print(model)

###################################################################################

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

###################################################################################

#plotting list
loss_list = []
time_list = []
t = 0

print("Start training")
# Train the model
total_step = len(train_loader)
for param in model.rnn.parameters():
        param.requires_grad = False
        
for epoch in range(num_epochs):
     
    for i, (images, labels) in enumerate(train_loader):
        freeze_layers
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.data[0]))
            loss_list.append(loss.data[0])
            time_list.append(t)
            t+=1

correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += predicted.eq(labels.data).sum()

print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

%matplotlib inline
import matplotlib.pyplot as plt
import np
plt.plot(np.asarray(time_list)[::4], np.asarray(loss_lis[::4]))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss for LSTM-CNN')
plt.show()

BiRNN(
  (rnn): LSTM(49, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (conv1): Conv2d(1, 8, kernel_size=(30, 30), stride=(1, 1))
  (conv2): Conv2d(1, 8, kernel_size=(30, 30), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=113, out_features=16, bias=True)
)
Start training


/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:137: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:138: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/60], Step [100/480], Loss: 2.0535
Epoch [1/60], Step [200/480], Loss: 1.8979
Epoch [1/60], Step [300/480], Loss: 1.5753
Epoch [1/60], Step [400/480], Loss: 1.5173
Epoch [2/60], Step [100/480], Loss: 1.2730
Epoch [2/60], Step [200/480], Loss: 1.0720
Epoch [2/60], Step [300/480], Loss: 0.8932
Epoch [2/60], Step [400/480], Loss: 0.9677
Epoch [3/60], Step [100/480], Loss: 1.2443
Epoch [3/60], Step [200/480], Loss: 1.1975
Epoch [3/60], Step [300/480], Loss: 1.1995
Epoch [3/60], Step [400/480], Loss: 0.9247
Epoch [4/60], Step [100/480], Loss: 1.0551
Epoch [4/60], Step [200/480], Loss: 1.0950
Epoch [4/60], Step [300/480], Loss: 0.7580
Epoch [4/60], Step [400/480], Loss: 1.0920
Epoch [5/60], Step [100/480], Loss: 0.7240
Epoch [5/60], Step [200/480], Loss: 1.3468
Epoch [5/60], Step [300/480], Loss: 0.9628
Epoch [5/60], Step [400/480], Loss: 1.0299
Epoch [6/60], Step [100/480], Loss: 1.2001
Epoch [6/60], Step [200/480], Loss: 0.8729
Epoch [6/60], Step [300/480], Loss: 0.9995
Epoch [6/60

Epoch [48/60], Step [100/480], Loss: 0.2894
Epoch [48/60], Step [200/480], Loss: 0.4848
Epoch [48/60], Step [300/480], Loss: 0.0883
Epoch [48/60], Step [400/480], Loss: 0.1394
Epoch [49/60], Step [100/480], Loss: 0.2683
Epoch [49/60], Step [200/480], Loss: 0.3525
Epoch [49/60], Step [300/480], Loss: 0.3126
Epoch [49/60], Step [400/480], Loss: 0.3505
Epoch [50/60], Step [100/480], Loss: 0.0153
Epoch [50/60], Step [200/480], Loss: 0.4051
Epoch [50/60], Step [300/480], Loss: 0.2377
Epoch [50/60], Step [400/480], Loss: 0.2398
Epoch [51/60], Step [100/480], Loss: 0.3488
Epoch [51/60], Step [200/480], Loss: 0.1367
Epoch [51/60], Step [300/480], Loss: 0.2963
Epoch [51/60], Step [400/480], Loss: 0.2329
Epoch [52/60], Step [100/480], Loss: 0.5665
Epoch [52/60], Step [200/480], Loss: 0.0363
Epoch [52/60], Step [300/480], Loss: 1.1696
Epoch [52/60], Step [400/480], Loss: 0.1998
Epoch [53/60], Step [100/480], Loss: 0.1116
Epoch [53/60], Step [200/480], Loss: 0.1049
Epoch [53/60], Step [300/480], L

ModuleNotFoundError: No module named 'np'

In [ ]:
#NOW FOR THE PSYCHOPHYSICS
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

exp_info = 'model_ring_FROZEN_HIRES2'
print (exp_info)

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.bmp' % (theta, B_theta)
    imageio.imwrite(uri = fname,im = im[:, :, 0], format = 'bmp')
    return fname

In [ ]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob

files = glob.glob('./tmp/*')
if not os.path.exists('./tmp'): #si le folder n'existe pas on le crée
        os.makedirs('./tmp')
        
        
test_length = 600
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()

ans_list = []
N_B_thetas = 15
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2).cuda()
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])

#cleanup temp
for f in files:
    os.remove(f)

In [ ]:
import pickle
pickle.dump(ans_list, open('./psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))

# Le convo

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=1, drop_last = True)

#test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4,shuffle=True,
                                             num_workers=1, drop_last = True)

In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 20)
        self.pool = nn.MaxPool2d(2,2)

        self.fc3 = nn.Linear(17496,1000)
        self.dropout = nn.Dropout(0.2)

        self.outlayer = nn.Linear(1000,16)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))

        x = x.view(x.size(0), -1) #reshape from conv to linear

        x = F.leaky_relu(self.fc3(x))
        x = self.dropout(x)

        x = self.outlayer(x)
        return x
        
model = Net()
print(model)

In [ ]:
import torch.optim as optim
import time

#unfreeze before starting
layers_microwave(model)

criterion = nn.CrossEntropyLoss() #loss criterion
optimizer = optim.SGD(params = model.parameters(),lr=0.001, momentum=0.9)
epochs = 2 #nbr of epochs per layer
model_size = 4 #nbr of layers

print_interval = 50 #prints every p_i*4
tempo = []
acc = []

start_time = time.time()
print("Started training")

for epoch in range(epochs):  # nbr epochs
    for batch_idx, (data, target) in enumerate(train_loader): #nbr batch,in,out
        data, target = Variable(data), Variable(target)
        #On resize pour la sortie


        #init l'entrainement
        optimizer.zero_grad()
        net_out = model(data)

        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

        #afficher la progression
        if batch_idx % print_interval == 0:
            #le print statement le plus illisible du monde
            print('Epoch: {} [{}/{} ({:.0f}%)]\t\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
    tempo.append(epoch)
    acc.append(loss.data[0])


print("Finished training in  %.3f seconds " % (time.time() - start_time))

In [ ]:
# delete
test_loss = 0
correct = 0
model = model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)

    net_out = model(data)
    
    #somme des pertes du batch
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1] #prediction
    correct += pred.eq(target.data).sum() #output du réseau

test_loss /= len(test_loader.dataset) #loss = loss/length set
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

# Convo psychophysique

In [ ]:
#j'ai augmenté les B_theta et la longueur du test pour lisser les courbes
exp_info = 'convo_model_HIRES'
print (exp_info)

In [ ]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

In [ ]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
from PIL import Image

test_length = 600
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()


ans_list = []
N_B_thetas = 15
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2)
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])


In [ ]:
import pickle
pickle.dump(ans_list, open('./psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))